In [3]:
import numpy as np
import os
import pandas as pd
import matplotlib as plt
import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import seaborn as sns
import glob
import re

We aim to calculate and rank the fold change of each Gene between the control and treatment groups to better understand the source of dispersion, which could be attributed to the effect of samples and sgRNA on Gene readcount.

In [50]:
!pwd

/Users/kanferg/Desktop/NIH_Youle/Colobration/Peroxisome_Project/AIPS_simulation


Loading table:

In [45]:
dfTall = pd.read_csv('dfTall.csv')
dfTall = dfTall.drop(columns='Unnamed: 0')
dfTall

,Gene,variable,value
0,AAAS,1_ctrl,14
1,AAAS,1_ctrl,37
2,ABCA8,1_ctrl,12
3,ABCA8,1_ctrl,42
4,ABCB4,1_ctrl,11
...,...,...,...
23845,ZNF502,6_activate,3
23846,ZNF563,6_activate,5
23847,ZNF607,6_activate,1
23848,ZNF607,6_activate,2


### Data structure:

Value - Read count per sgRNA (indexed numerically)\
Gene - Each sgRNA targets a unique gene; for example, sgRNA[0] and [1] target Gene AAAS\
Condition - 0 represents the read counts of the control, while 1 represents the read counts of the treated (Activated)\
Variable - Six samples, with three in the control group and three in the treated group

### Design Matrix

In [44]:
dfTall['Condition'] = 1
dfTall.loc[dfTall.variable.str.contains('ctrl'),'Condition'] = 0
idxCtrl = dfTall.loc[dfTall.Condition == 0,:].index
idxActive = dfTall.loc[dfTall.Condition == 1,:].index
dfTall_oneHot = pd.get_dummies(dfTall, columns = ['Condition','variable','Gene'])

arr = dfTall_oneHot.iloc[:].to_numpy() #(23850 rows × 1963 columns)
# design condtion
Xcond = arr[:,1:3] #(23850, 2)
# design Genes
Xgenes = arr[:,9:] #(46236, 1954)
# design samles
Xsamples = arr[:,3:9] #(46236, 6)

Y = arr[:,0]

arrDisp = Xsamples
ngenes = np.shape(Xgenes)[1]
nsamples = np.shape(Xsamples)[1]
ncondition = np.shape(Xcond)[1] #intercept
print(f'Xcond: {Xcond.shape}, Xgenes: {Xgenes.shape}, Xsamples: {Xsamples.shape}')
print(f'ncondition: {ncondition}, ngenes: {ngenes}, nsamples: {nsamples}')

Xcond: (23850, 2), Xgenes: (23850, 1954), Xsamples: (23850, 6)
ncondition: 2, ngenes: 1954, nsamples: 6
